# Visualizing PSD, SCV and Specified Frequency distributions

**Objective:**
visualizing global effects through PSD and SCV and local effects through specified frequency to introspect the electrophysiology data.

In [1]:
import sys
# locate your spectralCV so we have scv_funcs to use
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/notebooks_visualization/')

In [2]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'

In [3]:
# os.chdir('/Users/ldliao/Research/Projects/spectralCV/notebooks_visualization/')

In [4]:
# imports
import numpy as np
import scipy as sp
from scipy.stats import expon
import glob

import neurodsp as ndsp
from scv_funcs import lfpca
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'scv_funcs'

In [12]:
# bokeh imports
import bokeh
from bokeh.io import push_notebook, show, output_notebook, curdoc
from bokeh.io import *
from bokeh.plotting import figure
from bokeh.layouts import gridplot, row, column
from bokeh.models import ColumnDataSource, TapTool, Slider, Span, CustomJS
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [13]:
# import lfpca_vis

In [14]:
from bokeh.document import Document

In [15]:
# create bokeh document
doc = Document()
# type(lfpca_vis.modify_doc(doc))
# show(lfpca_vis.modify_doc(), notebook_handle=True)

In [26]:
def modify_doc(doc):

    # collect all the names of .npz files in the folder
#     lfpca_all_names = glob.glob("*.npz")
#     lfpca_all_names.sort()
    lfpca_all_names = ['move.npz', 'pre.npz', 'whole.npz']
    lfpca_all_names.sort()
    
    # loading the npz files 
    lfpca_all = {}
    for ind, lf in enumerate(lfpca_all_names):
        lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

    # initialize with the first lfpca object
    lf = lfpca_all[lfpca_all_names[0][:-4]]

    # grabbing channel count from psd
    chan_count, freq = lf.psd.shape

    # mapping all the channels
    DEFAULT_TICKERS = list(map(str, range(chan_count)))
    LF_TICKERS = [key for key in lfpca_all.keys()]

    # initializing values for frequency, psd, scv, histogram plot
    chan = 0
    select_freq = 10
    select_bin = 20
    freq_vals = lf.f_axis[1:]
    psd_vals = lf.psd[chan].T[1:]
    scv_vals = lf.scv[chan].T[1:]

    # creating a selector and slider
    lf_ticker = Select(value=lfpca_all_names[0][:-4], title='lf_condition', options=LF_TICKERS)
    ticker = Select(value=str(chan), title='channel', options=DEFAULT_TICKERS)
    freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")
    bin_slider = Slider(start=10, end=55, value=select_bin, step=5, title="Number of bins", callback_policy="mouseup")

    # create data and selection tools
    source = ColumnDataSource(data=dict(freq_vals=freq_vals, psd_vals=psd_vals, scv_vals=scv_vals))

    TOOLS = "help" #tapTool work in progress

    # setting up plots
    psd_plot = figure(tools=TOOLS, title='PSD', x_axis_type='log', y_axis_type='log')
    psd_plot.legend.location = 'top_left'
    psd_plot.xaxis.axis_label = 'Frequency (Hz)'
    psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
    psd_plot.grid.grid_line_alpha=0.3

    scv_plot = figure(tools=TOOLS, title='SCV', x_axis_type='log', y_axis_type='log')
    scv_plot.legend.location='top_left'
    scv_plot.xaxis.axis_label = 'Frequency (Hz)'
    scv_plot.yaxis.axis_label = '(Unitless)'
    scv_plot.grid.grid_line_alpha=0.3

    # create histogram frame
    hist_source = ColumnDataSource({'top': [], 'left': [], 'right': []})
    fit_hist_source = ColumnDataSource({'x': [], 'y': []})
    hist, edges = np.histogram(lf.spg[chan, select_freq, :], bins=select_bin, density=True)
    hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}

    # create fit line for the histogram
    rv = expon(scale=sp.stats.expon.fit(lf.spg[chan,select_freq,:],floc=0)[1])
    hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}
    fit_hist_source.data = {'x': edges, 'y': rv.pdf(edges)}

    hist_fig = figure(x_axis_label='Power', 
                      y_axis_label='Probability', background_fill_color="#E8DDCB")
    hist_fig.axis.visible = False
    hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(select_freq, lf.ks_pvals[chan, select_freq])


    # customize plot to psd
    def create_psd_plot(psd_plot, source):
        psd_plot.line('freq_vals', 'psd_vals', source=source, color='navy')
#         psd_plot.circle('freq_vals', 'psd_vals', source=source, size=5, color='darkgrey', alpha=0.2, 
#                         # set visual properties for selected glyphs
#                         selection_color="firebrick",
#                         # set visual properties for non-selected glyphs
#                         nonselection_fill_alpha=0.2,
#                         nonselection_fill_color="darkgrey",
#                         name='psd_circ')
    # customize plot to psd
    def create_scv_plot(scv_plot, source):
        fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
        scv_plot.add_glyph(source, fit_line)
        scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')
#         scv_plot.line(source['freq_vals'], np.ones(len(source['scv_vals'])), color='k')
#         scv_plot.circle('freq_vals', 'scv_vals', source=source, size=5, color='darkgrey', alpha=0.2, 
#                         # set visual properties for selected glyphs
#                         selection_color="firebrick",
#                         # set visual properties for non-selected glyphs
#                         nonselection_fill_alpha=0.2,
#                         nonselection_fill_color="darkgrey",
#                         name='scv_circ')

    # customize histogram
    def create_hist(hist_fig, hist_source):
        hist_fig.quad(top='top', bottom=0, left='left', right='right', fill_color="#036564", line_color="#033649", source=hist_source)

    # initializing plots
    create_psd_plot(psd_plot, source)
    create_scv_plot(scv_plot, source)
    vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
    vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
    psd_plot.add_layout(vline_psd)
    scv_plot.add_layout(vline_scv)
    create_hist(hist_fig, hist_source)
    fit_line = bokeh.models.glyphs.Line(x='x', y='y', line_width=8, line_alpha=0.7, line_color="#D95B43")
    hist_fig.add_glyph(fit_hist_source, fit_line)

    all_plots = gridplot([[psd_plot, scv_plot, hist_fig]], plot_width=300, plot_height=300)

    # set up connector spans
    freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                              span2 = vline_scv,
                                              slider = freq_slider),
                                              code = """span1.location = slider.value; 
                                                        span2.location = slider.value""")

    def update(attrname, old, new):
        # get current slider values
        chan = int(ticker.value)
        lf = lfpca_all[lf_ticker.value]
        select_freq = freq_slider.value
        select_bin = bin_slider.value

        # update data
        psd_vals = lf.psd[chan].T[1:]
        scv_vals = lf.scv[chan].T[1:]
        data = dict(freq_vals=freq_vals, psd_vals=psd_vals, scv_vals=scv_vals)
        # create a column data source for the plots to share
        source.data = data

        # update histogram and fit line
        hist, edges = np.histogram(lf.spg[chan, select_freq, :], bins=select_bin, density=True)
        rv = expon(scale=sp.stats.expon.fit(lf.spg[chan,select_freq,:],floc=0)[1])
        hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}
        fit_hist_source.data = {'x': edges, 'y': rv.pdf(edges)}
        create_psd_plot(psd_plot=psd_plot, source=source)
        create_scv_plot(scv_plot=scv_plot, source=source)
        hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(select_freq, lf.ks_pvals[chan, select_freq])
        create_hist(hist_fig=hist_fig, hist_source=hist_source)
        fit_line = bokeh.models.glyphs.Line(x='x', y='y', line_width=8, line_alpha=0.7, line_color="#D95B43")
        hist_fig.add_glyph(fit_hist_source, fit_line)

    # whenever a widget changes, the changes are tracked and histogram always updated
    for widget in [lf_ticker, ticker, bin_slider, freq_slider]:
        widget.on_change('value', update)

    # when selected value changes, take the following methods of actions
    # lf_ticker.on_change('value', lf_selection_change)
    # ticker.on_change('value', selection_change)

    # what to do when freq slider value changes
#     def freq_change(attr, old, new):
#         select_freq = source.selected.indices[0]
#         # update histogram and fit line
#         hist, edges = np.histogram(lf.spg[chan, select_freq, :], bins=select_bin, density=True)
#         rv = expon(scale=sp.stats.expon.fit(lf.spg[chan,select_freq,:],floc=0)[1])
#         hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}
#         fit_hist_source.data = {'x': edges, 'y': rv.pdf(edges)}
#         hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(select_freq, lf.ks_pvals[chan, select_freq])
#         create_hist(hist_fig=hist_fig, hist_source=hist_source)
#         fit_line = bokeh.models.glyphs.Line(x='x', y='y', line_width=8, line_alpha=0.7, line_color="#D95B43")
#         hist_fig.add_glyph(fit_hist_source, fit_line)
    
    # organize layout
    widgets = row(lf_ticker, ticker)
    sliders = row(freq_slider, bin_slider)
    layout = column(widgets, sliders, all_plots)
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
show(modify_doc, notebook_handle=True)
# curdoc().add_root(layout) - for .py file

### Comparing conditions

In [23]:
ls

move.npz        pre.npz         swim_plot.html  trial_info.npz  whole.npz


In [24]:
def modify_doc(doc):

    # collect all the names of .npz files in the folder
    lfpca_all_names = ['move.npz', 'pre.npz', 'whole.npz']
    lfpca_all_names.sort()
#     lfpca_all_names = glob.glob("*.npz")
#     lfpca_all_names.sort()
    
    # loading the npz files 
    lfpca_all = {}
    for ind, lf in enumerate(lfpca_all_names):
        lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

    # initialize with the first lfpca object
    lf1 = lfpca_all[lfpca_all_names[0][:-4]]
    lf2 = lfpca_all[lfpca_all_names[1][:-4]]
    
    # grabbing channel count from psd
    chan_count, freq = lf1.psd.shape

    # mapping all the channels
    CH_TICKERS = list(map(str, range(chan_count)))
    LF_TICKERS = [key for key in lfpca_all.keys()]
    
    def nix(val, lst):
        return [x for x in lst if x != val]

    # initializing values for frequency, psd, scv, histogram plot
    chan = 0
    select_freq = 10
    freq_vals_1 = lf1.f_axis[1:]
    freq_vals_2 = lf2.f_axis[1:]
    psd_vals_1 = lf1.psd[chan].T[1:]
    psd_vals_2 = lf2.psd[chan].T[1:]
    scv_vals_1 = lf1.scv[chan].T[1:] 
    scv_vals_2 = lf2.scv[chan].T[1:]

    # creating a selector and slider
    lf_ticker_1 = Select(value=lfpca_all_names[0][:-4], title='condition_1', 
                         options=nix(lfpca_all_names[1][:-4], LF_TICKERS))
    lf_ticker_2 = Select(value=lfpca_all_names[1][:-4], title='condition_2', 
                         options=nix(lfpca_all_names[0][:-4], LF_TICKERS))

    ch_ticker = Select(value=str(chan), title='channel', options=CH_TICKERS)
    freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")

    # create data and selection tools
    source_1 = ColumnDataSource(data=dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1))
    source_2 = ColumnDataSource(data=dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2))

    # TOOLS = "help" #tapTool work in progress

    # setting up plots
    psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log')
    psd_plot.legend.location = 'top_left'
    psd_plot.xaxis.axis_label = 'Frequency (Hz)'
    psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
    psd_plot.grid.grid_line_alpha=0.3

    scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log')
    scv_plot.legend.location='top_left'
    scv_plot.xaxis.axis_label = 'Frequency (Hz)'
    scv_plot.yaxis.axis_label = '(Unitless)'
    scv_plot.grid.grid_line_alpha=0.3

    # customize plot to psd
    def create_psd_plot(psd_plot, source_1, source_2):
        psd_plot.line('freq_vals_1', 'psd_vals_1',  source=source_1, 
                      color='navy', muted_color='navy', muted_alpha=0.2, 
                      legend='cond_1')
        psd_plot.line('freq_vals_2', 'psd_vals_2',  source=source_2, 
                      color="green", muted_color='green', muted_alpha=0.2, legend='cond_2')
        psd_plot.legend.location = "bottom_left"
        psd_plot.legend.click_policy="mute"
        
    # customize plot to psd
    def create_scv_plot(scv_plot, source_1, source_2):
        fit_line = bokeh.models.glyphs.Line(x='freq_vals_1', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
        scv_plot.add_glyph(source_1, fit_line)
        scv_plot.line('freq_vals_1', 'scv_vals_1',  source=source_1, 
                      line_color="navy", muted_alpha=0.2, legend='cond_1')
                      # legend=lf_ticker_1.value)
        scv_plot.line('freq_vals_2', 'scv_vals_2',  source=source_2, 
                      line_color="green", muted_alpha=0.2, legend='cond_2')
                      # legend=lf_ticker_2.value)
        scv_plot.legend.location = "bottom_left"
        scv_plot.legend.click_policy="mute"

    # initializing plots
    create_psd_plot(psd_plot, source_1, source_2)
    create_scv_plot(scv_plot, source_1, source_2)
    vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
    vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
    psd_plot.add_layout(vline_psd)
    scv_plot.add_layout(vline_scv)

    all_plots = gridplot([[psd_plot, scv_plot]], plot_width=450, plot_height=400)

    # set up connector spans
    freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                              span2 = vline_scv,
                                              slider = freq_slider),
                                              code = """span1.location = slider.value; 
                                                        span2.location = slider.value""")
#     def update(selected=None):
#         t1, t2 = lf_ticker_1.value, lf_ticker_2.value.value

#         data = get_data(t1, t2)
#         source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
#         source_static.data = source.data

#         update_stats(data, t1, t2)

#         corr.title.text = '%s returns vs. %s returns' % (t1, t2)
#         ts1.title.text, ts2.title.text = t1, t2

    def update(selected=None):
        # get current slider values
        chan = int(ch_ticker.value)
        lf1 = lfpca_all[lf_ticker_1.value]
        lf2 = lfpca_all[lf_ticker_2.value]
#         select_freq = freq_slider.value

        # update data
        freq_vals_1 = lf1.f_axis[1:]
        freq_vals_2 = lf2.f_axis[1:]
        psd_vals_1 = lf1.psd[chan].T[1:]
        psd_vals_2 = lf2.psd[chan].T[1:]
        scv_vals_1 = lf1.scv[chan].T[1:] 
        scv_vals_2 = lf2.scv[chan].T[1:]
        data_1 = dict(freq_vals_1=freq_vals_1, psd_vals_1=psd_vals_1, scv_vals_1=scv_vals_1)
        data_2 = dict(freq_vals_2=freq_vals_2, psd_vals_2=psd_vals_2, scv_vals_2=scv_vals_2)

        # create a column data source for the plots to share
        source_1.data = data_1
        source_2.data = data_2


#     # whenever a widget changes, the changes are tracked and histogram always updated
#     for widget in [lf_ticker, ticker, freq_slider]:
#         widget.on_change('value', update)

    def lf_ticker_1_change(attrname, old, new):
        lf_ticker_2.options = nix(new, LF_TICKERS)
        update()

    def lf_ticker_2_change(attrname, old, new):
        lf_ticker_1.options = nix(new, LF_TICKERS)
        update()
    
    def ch_ticker_change(attrname, old, new):
        update()
    
    lf_ticker_1.on_change('value', lf_ticker_1_change)
    lf_ticker_2.on_change('value', lf_ticker_2_change)
    ch_ticker.on_change('value', ch_ticker_change)

    # organize layout
    widgets = row(lf_ticker_1, lf_ticker_2, ch_ticker)
    sliders = row(freq_slider)
    layout = column(widgets, sliders, all_plots)
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
show(modify_doc, notebook_handle=True)
# curdoc().add_root(layout) - for .py file